# High performance pandas

´´´py

mask = (x > -5) & (y < -5)

df[df[mask]]

´´´ 

under the hood

tmp = (x > -5)
tmp2 = (y < -5)
mask = tmp & tmp2

use pd.eval(""), df.query() -> does element wise operations with numexpr

In [11]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.171621,1.072953,-0.656263,-0.024674,-0.402074,-0.001419,-1.743374,1.584496,-2.085512,-0.052577,...,-0.799176,-0.307223,0.391307,1.434631,2.394251,-1.410032,0.496211,-0.690535,0.743989,0.477275
1,0.736980,-1.456147,-0.730924,0.322530,0.883642,1.335024,0.362599,0.073098,1.555125,-0.640715,...,-0.174267,1.606671,1.695150,-2.279604,-0.962599,0.738291,0.327718,2.050804,-0.677776,-0.307132
2,0.160925,-0.067570,1.329889,-1.013781,-0.173770,0.134501,-1.036424,-1.271312,1.312510,-0.808666,...,-1.399011,0.630021,-1.854538,0.093013,-0.924833,-2.462231,0.151406,-0.522403,2.660795,0.010123
3,-0.036950,0.302604,-0.155371,0.529128,2.115415,2.407471,-1.416204,0.196162,-1.617763,0.066501,...,-0.319825,2.808097,0.758526,0.781059,-0.798526,-0.746724,2.137982,-0.670354,-1.398269,-0.567190
4,-0.469884,-1.261187,-0.244045,-0.477104,0.865259,-1.887970,-1.995649,-0.686765,-0.712253,-0.367094,...,1.007205,-0.429502,0.620863,0.115680,0.599667,-0.127200,1.848288,1.297058,0.518241,2.587802


In [12]:
df1.shape

(1000000, 100)

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [14]:
%timeit df1 + df2 + df3 + df4
%timeit pd.eval("df1+df2+df3+df4")

688 ms ± 38.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
243 ms ± 9.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [20]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,4,5,2
1,1,5,1
2,5,2,2
3,4,1,4
4,5,5,2
5,4,5,2


In [23]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,4,5,2,11
1,1,5,1,7
2,5,2,2,9
3,4,1,4,9
4,5,5,2,12
5,4,5,2,11


In [25]:
high = 10
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,4,5,2,11,True
1,1,5,1,7,False
2,5,2,2,9,False
3,4,1,4,9,False
4,5,5,2,12,True
5,4,5,2,11,True


In [27]:
# traditional way to filter from a dataframe
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
0,4,5,2,11
4,5,5,2,12
5,4,5,2,11


## Query
- filter using query

In [35]:
rolls.query("Sum > @high & Die1 == 4")

,Die1,Die2,Die3,Sum
0,4,5,2,11
5,4,5,2,11
